In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [16]:
# 归一化函数。
def max_min(X): # 这里传入的是一个矩阵，也就是一个像素点矩阵。
    return X/255  # 将每个像素点全部除以255,进行归一化处理。

In [17]:
# 这里是激活函数
def softmax_function(x):  # 这里传入的是一个矩阵
    max_data = np.max(x,axis = 1)  # 这里直接找出每一行的最大值。
    #print(max_data)
    x = x.T   # 这里要转置一下，十分重要！吃了大亏，因为上面那个max_data是行向量。
    ex = x - max_data# 这里直接把数据相减
    finish = np.exp(ex) / np.sum(np.exp(ex),axis = 0) # 这里由于矩阵已经转置过来了，就正常操作。
    return finish.T   #将最终的结果返回。
# 这里实在是太坑了，求和最大值全部都是行向量。
    

In [18]:
# 正向传播：
def sigmoid_forward(X,w1,b1):
    a1 = np.dot(X,w1) + b1  # 先将参数转化为线性模式,这里要注意的就是这个权重的问题。
    #print(a1)                   # 这里的参数调整有点微妙反正就是。
    z1 = sigmoid_function(a1)  # 转化为sigmoid形式。
    return z1,a1   # 返回一个矩阵。

In [19]:
# 计算损失值的函数：
def loss_function(y,y_pre):  # 这里只要计算输出层的那个损失值即可，也就是说是softmax函数那一层的东西。
    # 这里传入的是one-hor编码。
    loss = - np.mean( np.log(y * y_pre + 1e-7 ) )  # 直接使用one-hot编码来计算损失值，比for循环快,。
    return loss

In [20]:
def softmax_reverse(y,y_pre,num,z1):  # 首先传入的是一个one-hot编码 这个十分重要。然后就是样本的总数和矩阵。
    dl_dz2 = y_pre - y   # 这里是损失函数对z2的求导。
    new_w = np.dot(z1.T,dl_dz2) / num  # 这里要使用的是链式求导法则，最终可以得到偏导数的矩阵。
    db = np.sum(dl_dz2,axis = 0) / num # 在这里表示函数对偏置项的偏导数,这里直接列项求和然后去平均。
    return new_w,db   # 这里要返回3个值。

In [21]:
# 导入数据。、
data = pd.read_csv('./softmax/train.csv')  # 将数据储存。
[n,m] = data.shape  # 查看数据集的大小。
print(n,m)
soft_x = np.array(data.iloc[: , 2:m])  # 将特征全部提取出来。
soft_y = np.array(data.iloc[: , 1:2]).T
soft_y = np.squeeze(soft_y)  # 将y转化为一维矩阵便于后续操作。
print(soft_x.shape)  # 查看数据有没有导入正确。
print(soft_y)
print(soft_x.shape)

20000 786
(20000, 784)
[0 4 1 ... 4 2 5]
(20000, 784)


In [22]:
[n,m] = soft_x.shape
print(n,m)      # 将样本的数据提取出来。
Z_data = max_min(soft_x)  # 将数据进行归一化。
#print(Z_data) # 这里Z_data表示的就是标准化后的数据。
one_hot_data = np.eye(10)[soft_y]
print(one_hot_data)  # 转化为one-hot编码。

20000 784
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [28]:
# 初始化。
np.random.seed(11)
w = 0.01*np.random.rand(784,10)
b = np.zeros(10)
cnt = 200
alpha = 0.01
loss = np.zeros(cnt)
for i in range(100):
    a = np.dot(Z_data,w) + b
    z = softmax_function(a)
    loss[i] = loss_function(one_hot_data,z)
    dw,db = softmax_reverse(one_hot_data,z,20000,Z_data)
    w -= alpha*dw
    b -= alpha*db
    
    

In [29]:
#print(loss)
print(w)
print(b)

[[0.0018027  0.00019475 0.00463219 ... 0.00487372 0.00941807 0.00850795]
 [0.00729964 0.00108736 0.00893904 ... 0.00116737 0.00316367 0.00157912]
 [0.0075898  0.00818275 0.00344624 ... 0.00599543 0.00055674 0.00479797]
 ...
 [0.00766954 0.00756178 0.0084     ... 0.0030715  0.00761515 0.0036372 ]
 [0.00450878 0.00482176 0.00179269 ... 0.00507    0.0088959  0.00685645]
 [0.00654547 0.0022641  0.00423949 ... 0.00208067 0.00214611 0.00110415]]
[-0.01142359  0.0234134  -0.00499789 -0.00304097  0.00340722  0.00072675
 -0.00221601  0.00672673 -0.01245111 -0.00014454]


In [30]:
# 接下来就是对测试集进行验证。
test_data = pd.read_csv('./softmax/test_no_answer.csv')

In [31]:
#print(test_data)
[n,m] = test_data.shape
test_x = test_data.iloc[:,1:m]   # 提取数据集。
print(test_x.shape)
test_x = np.array(test_x)  # 这里转化为矩阵的形式。
Z_test = max_min(test_x)   # 这里将数据归一化。
print(Z_test)

(2000, 784)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [32]:
test_a = np.dot(test_x,w) + b
test_z = softmax_function(test_a)

In [33]:
finish = np.argmax(test_z,axis = 1)

In [34]:
print(finish)

[3 1 0 ... 9 3 6]


In [35]:
finish = pd.DataFrame(finish)
finish.index = np.arange(1, len(finish)+1) 

In [36]:
finish.to_csv('./1111.csv')

In [37]:
# def softmax_derivative(Q):
#     x=softmax_function(Q)
#     s=x.reshape(-1,1)
#     return (np.diagflat(s) - np.dot(s, s.T))

In [38]:
print(Z_data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [40]:
Z_data = pd.DataFrame(Z_data)
Z_data.to_csv('./标准化softmax.csv')